## 1.2 신경망의 추론

## 1.2.1 신경망 추론 전체 그림

### 완전 연결계층(Fully Connected Layer)
신경망(Neural Network)의 중요한 구성 요소 중 하나로, 모든 입력 노드가 모든 출력 노드와 연결되어 있는 계층을 의미한다.  
이를 Dense Layer라고도 부른다.  
완전 연결계층은 주로 다층 퍼셉트론(Multi-Layer Perceptron, MLP)과 같은 신경망 구조에서 사용된다.  
#### 주요 특징
1. 완전 연결 : 각 입력 노드는 모든 출력 노드와 연결되어 있다. 따라서 각 출력 노드는 입력 노드의 가중 합을 통해 계산된다.  
2. 가중치($W$)와 바이어스($b$) : 각 연결은 고유한 가중치를 가지며, 각 출력 노드는 고유한 바이어스를 가진다.  
3. 활성화 함수 : 출력 노드의 값은 활성화 함수를 통해 결정된다.  
4. 출력 : 출력 값은 입력 노드의 가중 합과 바이어스를 더한 후 활성화 함수를 적용한 결과이다.  

### 행렬의 형상 확인
하나의 샘플 데이터 대상이 아닌 학습에서 다수의 샘플 데이터를 처리하기 위해서는 입력 데이터가 일치해야 한다.  
그 이유로는 형상에 대응하는 차원 원소 수가 일치해야하는데 이를 활용하기 위해서는 데이터의 일반화가 필요하다(?)

In [3]:
'''
완전연결계층에 의한 변환의 미니 배치버전 // 선형 함수
'''
import numpy as np
W1 = np.random.randn(2,4) # 가중치
b1 = np.random.randn(4) # 편향
x = np.random.randn(10, 2)
h = np.matmul(x, W1) + b1
print(h)

[[-1.18200168e+00  3.28390151e-01  3.90250938e-02 -1.28186664e+00]
 [-2.13529284e+00 -4.76495172e+00  1.24745964e+00 -1.36748408e+00]
 [-1.57523245e+00 -1.96648112e+00  3.98731785e-02 -1.87316283e+00]
 [-7.59694053e-01  3.05661929e+00  7.14873274e-01  1.09265536e-01]
 [-1.18690488e+00  5.38975334e-01  6.52994451e-01 -6.03289966e-01]
 [-2.54593708e-01  5.64215760e+00 -2.15873685e-01 -1.69887589e-01]
 [-8.62625773e-01  2.46311748e+00  7.33579630e-01 -2.48598234e-02]
 [-1.51273298e+00 -1.27725404e+00  8.72596912e-01 -8.48666389e-01]
 [-1.27646938e+00  1.63162281e-03  6.15583579e-01 -7.79980346e-01]
 [-1.47871411e+00 -1.05036188e+00  9.45315844e-01 -7.16184957e-01]]


In [6]:
# 시그모이드 함수를 활용하면 비선형 데이터를 나타낼 수 있고, 이는 활성화 함수 중 하나를 나타낸다.
def sifmoid(x) : 
    return 1/ (1 + np.exp(-x))

## 1.2.2 계층으로 클래스화 및 순전파 구현
책에서는 완전연결계층을 Affine 계층으로, 시그모이드 함수에 의한 변환은 Sigmoid 계층으로 구현 하였음  
신경망 추론과정에서 하는 처리를 신경망의 순전파(forward propagation)에 해당된다.  
순전파란 말 그대로 입력층에서 출력층으로 향하는 전파를 나타낸다. 반대되는 개념으로는 역전파(back propagation)이다.

#### 책의 계층 구현시 '구현 규칙'  
모든 계층은 forward() 와 backward() 메서드를 가진다.  
모든 계층은 인스턴스 변수인 params와 grads를 가진다.  
params는 가중치와 편향과 같은 매개변수를 담는 리스트(list)를 나타낸다.  
grads는 params에 저장된 각 매개변수에 대응하여, 해당 매개변수의 기울기를 보관하는 리스트를 나타낸다.  

In [10]:
# 시그모이드(Sigmoid) 레이어 구현
class Sigmoid:
    '''Sigmoid Layer class
    
    Sigmoid layer에는 학습하는 params가 따로 없으므로 
    인스턴스 변수인 params는 빈 리스트로 초기화
    
    '''
    def __init__(self):
        self.params = []
    
    def forward(self, x):
        """순전파(forward propagation) 메서드
        Args:
            x(ndarray): 입력으로 들어오는 값
        Returns:
            Sigmoid 활성화 값
        """
        return 1 / (1 + np.exp(-x))
    
# 완전연결계층(Affine) 구현
class Affine:
    '''FC layer'''
    def __init__(self, W, b):
        """
        Args: 
            W(ndarray): 가중치(weight)
            b(ndarray): 편향(bias)
        """
        self.params = [W, b]
        
    def forward(self, x):
        """순전파(forward propagation) 메서드
        Args:
            x(ndarray): 입력으로 들어오는 값
        Returns:
            out(ndarray): Wx + b
        """
        W, b = self.params
        out = np.matmul(x, W) + b
        return out

class TwoLayerNet:
    def __init__(self, input_size, hidden_size, output_size):
        I, H, O = input_size, hidden_size, output_size
        
        # 가중치와 편향 초기화
        # input -> hidden
        W1 = np.random.randn(I, H)  
        b1 = np.random.randn(H)
         # hidden -> output
        W2 = np.random.randn(H, O) 
        b2 = np.random.randn(O)
        
        # 레이어 생성
        self.layers = [
            Affine(W1, b1),
            Sigmoid(),
            Affine(W2, b2)
        ]
        
        # 모든 가중치를 리스트에 모은다.
        self.parmas = [layer.params for layer in self.layers]
        # self.params = []
        # for layer in self.layers:
        #     self.params += layer.params
        
    def predict(self, x):
        for layer in self.layers:
            x = layer.forward(x)
        return x

In [18]:
x = np.random.randn(10, 2) # 10개의 샘플이 2개의 특성을 가지고 있다라고 생각하면 된다
model = TwoLayerNet(2, 4, 3) # 2개의 특성을 가지고, 4개의 은닉층과 3개의 클래스를 분류한다.
s = model.predict(x)

print(f'input:\n{x}')
print(f'predict :\n{s}')

input:
[[-1.3416326   1.20690452]
 [-1.23226156 -0.42801423]
 [-1.58535409 -0.89558028]
 [ 0.59670275 -0.7675958 ]
 [ 0.84956351  1.26291406]
 [ 1.49416648 -1.17919111]
 [-0.07371392 -0.20913332]
 [-2.35206093  0.67837226]
 [ 0.07313945  1.55841736]
 [-0.97124501 -1.7813746 ]]
predict :
[[ 0.15547755  0.96468655 -0.41992304]
 [ 0.02010249  1.56222132  0.17393831]
 [-0.0204302   1.61344758  0.28029711]
 [-0.08696468  1.91235805  0.6590352 ]
 [ 0.30764588  1.31991877 -0.23993685]
 [-0.21731824  2.11599916  1.00698894]
 [ 0.04114777  1.68736734  0.27600336]
 [ 0.02766646  1.02219403 -0.32087504]
 [ 0.31253265  1.04542305 -0.43910746]
 [-0.16131569  1.84865069  0.7648478 ]]
